In [1]:
import matplotlib.pyplot as plt
from ekpy import control
from ekpy.control.instruments import keysight81150a, keysightdsox3024a
from PVloop import FEHysteresis
import numpy as np
import time

2


In [2]:
rm = control.ResourceManager()
rm.list_resources()

('GPIB0::8::INSTR', 'USB0::0x0957::0x17A6::MY63080078::0::INSTR')

In [3]:
#scope = rm.open_resource('USB0::0x0957::0x17A6::MY63080078::INSTR')
wavegen = rm.open_resource('GPIB0::8::INSTR')

In [4]:
keysightdsox3024a.idn(scope)

'AGILENT TECHNOLOGIES,DSO-X 3024A,MY63080078,02.65.2021030741\n'

In [4]:
keysight81150a.idn(wavegen)

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [5]:
wavegen.write(":VOLT1 :OFFS1")

15

In [9]:
wavegen.write(":SOUR:FUNC{}:{}".format('1', 'SQU'))
wavegen.write(":SOUR:FREQ{} {}".format('1', '250000'))
wavegen.write(":VOLT1 :OFFS1")

15

In [5]:
def wakeup_sherry(wavegen, channel='1', num_cycles='1e8', voltage='2', pulse_width='2e-6'):
    """
    This program will make a bipolar pulse and then configure it accordingly
    wavegen (pyvisa.resources.gpib.GPIBInstrument): Keysight 81150a
    channel (str): channel to output too ['1', '2']

    """
    basic_bipolar_pulse = [1,1,-1,-1] #basic pulse train
    keysight81150a.create_arb_wf(wavegen, basic_bipolar_pulse)
    frequency = 1/(2*float(pulse_width))
    keysight81150a.configure_arb_wf(wavegen, channel, freq=frequency, gain=voltage)
    keysight81150a.enable_output(wavegen, channel)
    total_wait_time = int(float(num_cycles)*(2* float(pulse_width)))
    time.sleep(total_wait_time)
    keysight81150a.enable_output(wavegen, channel, False)

In [6]:
wakeup_sherry(wavegen)

In [12]:
keysight81150a.enable_output(wavegen)

In [8]:
exp_fe = FEHysteresis(wavegen,scope)

# specify run function (if desired)
# options are: run_function aka i havent added a different one


# config data path
exp_fe.config_path('./newsampleworksradiant')

In [9]:
voltages = ['1']
scan_params = {
    'voltage':voltages
}

run_function_args = {
    'wavegen':wavegen,
    'scope':scope,
    'initial_delay':'5e-5',
    'pulse_delay':'5e-6',
    'freq': '10',
    'voltage':'2',
    'capacitor_area':'4e-8',
    'thickness':'10e-9',
    'permittivity':'30',
	'amplification':'8',
}

order = ['voltage',]

In [11]:
exp_fe.n_param_scan(scan_params, run_function_args, order, ntrials=5, plot=False)

done.
